# Langchain Retrievers

* Wikipedia Retriever
* Vector Store Retriever
* MMR
* Multi-Query Retriever
* Contexual Compression Retriever

In [ ]:
# !pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

## Wikipedia Retriever

In [1]:
from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [2]:
# Define your query
query = "the difference between indian, pakistani and bangladeshi culture, food, and language"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [3]:
docs

[Document(metadata={'title': 'Culture of Bangladesh', 'summary': 'The culture of Bangladesh is intertwined with the culture of the Bengal region of the Indian subcontinent. It has evolved over the centuries and encompasses the cultural diversity of several social groups of Bangladesh. The Bengal Renaissance of the 18th  early 19th centuries, noted Bengali writers, saints, authors, scientists, researchers, thinkers, music composers, painters,  film-makers have played a significant role in the development of Bengali culture. The culture of Bangladesh is deeply intertwined with the culture of the Bengal region. Basically, Bengali culture refers to the culture of Bangladesh. The Bengal Renaissance contained the seeds of a nascent political Indian nationalism which was the precursor in many ways to modern Indian artistic cultural expression.\nAccording to M. Nazrul Islam Tamij, a human rights activist and chairman of the National Human Rights Society (NHRS), human rights are the most import

In [4]:
# Print retrieved content
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


--- Result 1 ---
Content:
The culture of Bangladesh is intertwined with the culture of the Bengal region of the Indian subcontinent. It has evolved over the centuries and encompasses the cultural diversity of several social groups of Bangladesh. The Bengal Renaissance of the 18th  early 19th centuries, noted Bengali writers, saints, authors, scientists, researchers, thinkers, music composers, painters,  film-makers have played a significant role in the development of Bengali culture. The culture of Bangladesh is deeply intertwined with the culture of the Bengal region. Basically, Bengali culture refers to the culture of Bangladesh. The Bengal Renaissance contained the seeds of a nascent political Indian nationalism which was the precursor in many ways to modern Indian artistic cultural expression.
According to M. Nazrul Islam Tamij, a human rights activist and chairman of the National Human Rights Society (NHRS), human rights are the most important part of Bengali culture, and it play

## Vector Store Retriever

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [9]:
# Step 2: Initialize embedding model
embedding_model = OpenAIEmbeddings()

# Step 3: Create Chroma vector store in memory
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="my_collection"
)

In [10]:
# Step 4: Convert vectorstore into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [11]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [12]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


In [13]:
# We can also use the built-in similarity search method of the vectorstore to find similar documents. But it's not flexible like the retriever.
results = vectorstore.similarity_search(query, k=2)

for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
LangChain helps developers build LLM applications easily.


## Maximum Marginal Relevance (MMR)

In [14]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [16]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [17]:
query = "What is langchain?"
results = retriever.invoke(query)

In [18]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


## Multiquery Retriever

In [25]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [26]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
    Document(page_content="Stretching daily improves flexibility, reduces injury risk, and promotes circulation.", metadata={"source": "H6"}),
    Document(page_content="Limiting screen time before bed enhances melatonin production and sleep quality.", metadata={"source": "H7"}),
    Document(page_content="Strength training increases bone density and supports long-term mobility.", metadata={"source": "H8"}),
    Document(page_content="High-fiber diets support digestive health and regulate blood sugar levels.", metadata={"source": "H9"}),
    Document(page_content="Laughter reduces stress hormones and boosts immune response.", metadata={"source": "H10"}),
]

In [27]:
# Initialize OpenAI embeddings
embedding_model = OpenAIEmbeddings()

# Create FAISS vector store
vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

In [ ]:
# Create retrievers
similarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # returns exactly 5 results

In [ ]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}), # returns one extra result
    llm=ChatOpenAI(model="gpt-4.1-mini")
)

In [30]:
# Query
query = "How to improve energy levels and maintain balance?"

In [31]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [32]:
print("SIMPLE RETRIEVER RESULTS")
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

print("MULTI-QUERY RETRIEVER RESULTS")
for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

SIMPLE RETRIEVER RESULTS

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Stretching daily improves flexibility, reduces injury risk, and promotes circulation.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of depression.

--- Result 5 ---
Deep sleep is crucial for cellular repair and emotional regulation.
******************************************************************************************************************************************************
MULTI-QUERY RETRIEVER RESULTS

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Stretching daily improves flexibility, reduces injury risk, and promotes circulation.

--- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Resu

## Contextual Compression Retriever

In [34]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [35]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [36]:
# Create a FAISS vector store from the documents
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [37]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [38]:
# Set up the compressor using an LLM
llm = ChatOpenAI(model="gpt-4o-mini")
compressor = LLMChainExtractor.from_llm(llm)

In [39]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [40]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [41]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
Photosynthesis does not occur in animal cells.
